In [ ]:
#===============Cefra-seq========================
import os
import numpy as np
from Bio import SeqIO
from collections import Counter
from itertools import product
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import classification_report, matthews_corrcoef, accuracy_score
from sklearn.feature_extraction.text import TfidfTransformer
from imblearn.over_sampling import SMOTE

# تابع برای استخراج ویژگی‌های TF (k-mer)
def extract_tf_features(sequences, k=5):
    kmer_dict = {''.join(n): i for i, n in enumerate(product('ACGT', repeat=k))}
    features = []

    for seq in sequences:
        kmers = [seq[i:i+k] for i in range(len(seq)-k+1)]
        kmer_count = Counter(kmers)
        feature_vector = np.zeros(len(kmer_dict))
        for kmer, count in kmer_count.items():
            if kmer in kmer_dict:
                feature_vector[kmer_dict[kmer]] = count / len(kmers)
        features.append(feature_vector)
    
    return np.array(features)

# تابع برای استخراج Distance-based Subsequence Profiles
def extract_distance_based_features(sequences, max_k=8):
    patterns = [''.join(p) for p in product('ACGT', repeat=2)]  # 16 ترکیب
    features = []
    
    for seq in sequences:
        feature_vector = np.zeros((max_k + 1) * len(patterns))  # 9 * 16 = 144
        for k in range(max_k + 1):
            for i, pattern in enumerate(patterns):
                count = 0
                for j in range(len(seq) - k - 2):
                    if seq[j] == pattern[0] and seq[j + k + 1] == pattern[1]:
                        count += 1
                feature_vector[k * len(patterns) + i] = count / (len(seq) - k - 1) if len(seq) - k - 1 > 0 else 0
        features.append(feature_vector)
    
    return np.array(features)

# فایل FASTA
fasta_file = "F:\\New Version\\Data\\cefra-seq\\cefra_seq_cDNA_screened.fa"

# خواندن توالی‌ها و اطلاعات مکان‌یابی
sequences = []
locations = []

for record in SeqIO.parse(fasta_file, "fasta"):
    sequences.append(str(record.seq))
    description = record.description
    dist_str = description.split("dist:")[1]
    dist_values = [float(x) for x in dist_str.split("_")]
    locations.append(dist_values)

locations = np.array(locations)
y = np.argmax(locations, axis=1)
print("تعداد نمونه‌ها در هر کلاس (کل داده‌ها):", np.bincount(y))

# استخراج ویژگی‌های TF
tf_features = extract_tf_features(sequences, k=5)

# تبدیل TF به TF-IDF
tfidf = TfidfTransformer()
tfidf_features = tfidf.fit_transform(tf_features).toarray()

# استخراج Distance-based Subsequence Profiles
distance_features = extract_distance_based_features(sequences, max_k=8)

# بررسی تعداد نمونه‌ها
print("تعداد نمونه‌ها در TF-IDF:", tfidf_features.shape[0])
print("تعداد نمونه‌ها در Distance-based:", distance_features.shape[0])

# ترکیب ویژگی‌ها (بدون PPI)
X = np.hstack((tfidf_features, distance_features))
print("شکل ماتریس ویژگی‌ها:", X.shape)

# نرمال‌سازی ویژگی‌ها
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# اعمال SMOTE برای تعادل داده‌ها
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X_scaled, y)
print("تعداد نمونه‌ها در هر کلاس قبل از SMOTE:", np.bincount(y))
print("تعداد نمونه‌ها در هر کلاس بعد از SMOTE:", np.bincount(y_balanced))

# بهینه‌سازی SVM با Grid Search
param_grid = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto', 0.1, 1]}
svm = SVC(kernel='rbf', random_state=42)
grid = GridSearchCV(svm, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid.fit(X_balanced, y_balanced)
print("بهترین پارامترها:", grid.best_params_)
print("بهترین F1-score در Grid Search:", grid.best_score_)

# استفاده از بهترین مدل برای ارزیابی
best_svm = grid.best_estimator_

# پیش‌بینی‌ها با cross-validation
y_pred = cross_val_predict(best_svm, X_balanced, y_balanced, cv=10)

# محاسبه معیارهای کلی
f1_macro = cross_val_score(best_svm, X_balanced, y_balanced, cv=10, scoring='f1_macro').mean()
accuracy = accuracy_score(y_balanced, y_pred)
mcc = matthews_corrcoef(y_balanced, y_pred)

print("\nمعیارهای کلی:")
print(f"میانگین F1-score (ماکرو) با 10-fold cross-validation: {f1_macro:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"MCC: {mcc:.4f}")

# گزارش معیارها برای هر کلاس
class_names = ['Cytosol', 'Nucleus', 'Membrane', 'Insoluble']
print("\nگزارش معیارها برای هر کلاس:")
print(classification_report(y_balanced, y_pred, target_names=class_names))

# محاسبه MCC برای هر کلاس
print("\nMCC برای هر کلاس:")
for i, class_name in enumerate(class_names):
    y_true_binary = (y_balanced == i).astype(int)
    y_pred_binary = (y_pred == i).astype(int)
    mcc_class = matthews_corrcoef(y_true_binary, y_pred_binary)
    print(f"{class_name}: {mcc_class:.4f}")

In [3]:
#=================Cefra-seq with PPI==================
import os
import numpy as np
from Bio import SeqIO
from collections import Counter
from itertools import product
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import classification_report, matthews_corrcoef, accuracy_score
from sklearn.feature_extraction.text import TfidfTransformer
from imblearn.over_sampling import SMOTE
import pickle
from sklearn.decomposition import PCA

# تابع پیش‌پردازش داده‌ها
def process_cefra_data():
    print("Start processing CeFra-Seq Data phase")

    # خواندن توالی‌ها و اطلاعات مکان‌یابی
    fasta_file = "F:\\New Version\\Data\\cefra-seq\\cefra_seq_cDNA_screened.fa"
    X = []  # توالی‌ها
    y = []  # برچسب‌ها
    gene_ids = []  # شناسه‌های ژن
    all_gene_ids = []  # لیست اولیه ژن‌ها

    # ذخیره لیست اولیه ژن‌ها
    for record in SeqIO.parse(fasta_file, "fasta"):
        all_gene_ids.append(record.id)

    # خواندن دوباره برای فیلتر کردن
    for record in SeqIO.parse(fasta_file, "fasta"):
        description = record.description
        dist_str = description.split("dist:")[1]
        dist_values = [float(x) for x in dist_str.split("_")]
        dist_values = np.array(dist_values)
        dist_values = dist_values / dist_values.sum()
        # بررسی تفاوت بین بیشترین و دومین مقدار
        sorted_values = np.sort(dist_values)[::-1]
        diff = sorted_values[0] - sorted_values[1]
        if diff > 0.1:  # آستانه برای حذف نمونه‌های مبهم
            label = int(np.argmax(dist_values))
            y.append(label)
            gene_ids.append(record.id)
            X.append(str(record.seq))

    print("Number of gene ids:", len(gene_ids))
    new = list(set(gene_ids))
    print("Unique gene ids:", len(new))
    print("Number of all mRNA's:", len(X))
    print("Length of y:", len(y))
    print("چند شناسه ژن اول از CeFra-Seq:", gene_ids[:5])

    # شمارش تعداد نمونه‌ها در هر کلاس
    cytosol = 0
    nucleus = 0
    membrane = 0
    insoluble = 0
    for item in y:
        if item == 0:
            cytosol += 1
        elif item == 1:
            nucleus += 1
        elif item == 2:
            membrane += 1
        elif item == 3:
            insoluble += 1
    print("Cytosol:", cytosol, "Nucleus:", nucleus, "Membrane:", membrane, "Insoluble:", insoluble)

    # بدون فیلتر کردن برای Homo sapiens
    gene_info = [[gen, None, None] for gen in gene_ids]

    # تبدیل به آرایه NumPy
    X = np.asarray(X)
    y = np.asarray(y, dtype=np.int32)
    print("Shape of X DATA:", np.shape(X))
    print("Shape of y DATA:", np.shape(y))

    # ذخیره داده‌ها
    with open("C:\\Users\\Ali\\Desktop\\Grok\\cefra_sequences.pkl", "wb") as fp:
        pickle.dump(X, fp)
    with open("C:\\Users\\Ali\\Desktop\\Grok\\cefra_targets.pkl", "wb") as fp:
        pickle.dump(y, fp)
    with open("C:\\Users\\Ali\\Desktop\\Grok\\cefra_gene_info.pkl", "wb") as fp:
        pickle.dump(gene_info, fp)

    return X, y, gene_info, gene_ids, all_gene_ids

# تابع برای استخراج ویژگی‌های TF (k-mer)
def extract_tf_features(sequences, k=5):
    kmer_dict = {''.join(n): i for i, n in enumerate(product('ACGT', repeat=k))}
    features = []
    for seq in sequences:
        kmers = [seq[i:i+k] for i in range(len(seq)-k+1)]
        kmer_count = Counter(kmers)
        feature_vector = np.zeros(len(kmer_dict))
        for kmer, count in kmer_count.items():
            if kmer in kmer_dict:
                feature_vector[kmer_dict[kmer]] = count / len(kmers)
        features.append(feature_vector)
    return np.array(features)

# تابع برای استخراج Distance-based Subsequence Profiles
def extract_distance_based_features(sequences, max_k=8):
    patterns = [''.join(p) for p in product('ACGT', repeat=2)]
    features = []
    for seq in sequences:
        feature_vector = np.zeros((max_k + 1) * len(patterns))
        for k in range(max_k + 1):
            for i, pattern in enumerate(patterns):
                count = 0
                for j in range(len(seq) - k - 2):
                    if seq[j] == pattern[0] and seq[j + k + 1] == pattern[1]:
                        count += 1
                feature_vector[k * len(patterns) + i] = count / (len(seq) - k - 1) if len(seq) - k - 1 > 0 else 0
        features.append(feature_vector)
    return np.array(features)

# پیش‌پردازش داده‌ها
sequences, y, gene_info, gene_ids, all_gene_ids = process_cefra_data()

# چاپ تعداد نمونه‌ها در هر کلاس
print("تعداد نمونه‌ها در هر کلاس (کل داده‌ها):", np.bincount(y))

# بررسی اینکه آیا داده‌ای وجود دارد
if len(sequences) == 0:
    raise ValueError("هیچ داده‌ای وجود ندارد. لطفاً داده‌های ورودی را بررسی کنید.")

# استخراج ویژگی‌های TF
tf_features = extract_tf_features(sequences, k=5)

# تبدیل TF به TF-IDF
tfidf = TfidfTransformer()
tfidf_features = tfidf.fit_transform(tf_features).toarray()

# استخراج Distance-based Subsequence Profiles
distance_features = extract_distance_based_features(sequences, max_k=8)

# بررسی تعداد نمونه‌ها
print("تعداد نمونه‌ها در TF-IDF:", tfidf_features.shape[0])
print("تعداد نمونه‌ها در Distance-based:", distance_features.shape[0])

# بارگذاری ماتریس PPI
ppi_matrix = np.load("F:\\New Version\\PPIdata\\ppiMatrixScoress.npy")
print("شکل ماتریس PPI:", ppi_matrix.shape)

# فیلتر کردن ماتریس PPI بر اساس ژن‌های باقی‌مانده
indices = [all_gene_ids.index(gene_id) for gene_id in gene_ids]
ppi_matrix_filtered = ppi_matrix[indices, :][:, indices]
print("شکل ماتریس PPI بعد از فیلتر کردن:", ppi_matrix_filtered.shape)

# کاهش ابعاد با PCA
pca = PCA(n_components=500)
ppi_features = pca.fit_transform(ppi_matrix_filtered)
print("شکل ویژگی‌های PPI بعد از PCA:", ppi_features.shape)

# ترکیب ویژگی‌ها (TF-IDF + Distance-based + PPI)
X_with_ppi = np.hstack((tfidf_features, distance_features, ppi_features))
print("شکل ماتریس ویژگی‌ها با PPI:", X_with_ppi.shape)

# نرمال‌سازی ویژگی‌ها
scaler = StandardScaler()
X_scaled_with_ppi = scaler.fit_transform(X_with_ppi)

# اعمال SMOTE برای تعادل داده‌ها
smote = SMOTE(random_state=42)
X_balanced_with_ppi, y_balanced = smote.fit_resample(X_scaled_with_ppi, y)
print("تعداد نمونه‌ها در هر کلاس بعد از SMOTE (با PPI):", np.bincount(y_balanced))

# بهینه‌سازی SVM با Grid Search
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': ['scale', 'auto', 0.01, 0.1, 1, 10]}
svm = SVC(kernel='rbf', random_state=42)
grid = GridSearchCV(svm, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid.fit(X_balanced_with_ppi, y_balanced)
print("بهترین پارامترها:", grid.best_params_)
print("بهترین F1-score در Grid Search:", grid.best_score_)

# استفاده از بهترین مدل برای ارزیابی
best_svm = grid.best_estimator_

# پیش‌بینی‌ها با cross-validation
y_pred = cross_val_predict(best_svm, X_balanced_with_ppi, y_balanced, cv=10)

# محاسبه معیارهای کلی
f1_macro = cross_val_score(best_svm, X_balanced_with_ppi, y_balanced, cv=10, scoring='f1_macro').mean()
accuracy = accuracy_score(y_balanced, y_pred)
mcc = matthews_corrcoef(y_balanced, y_pred)

print("\nمعیارهای کلی:")
print(f"میانگین F1-score (ماکرو) با 10-fold cross-validation: {f1_macro:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"MCC: {mcc:.4f}")

# گزارش معیارها برای هر کلاس
class_names = ['Cytosol', 'Nucleus', 'Membrane', 'Insoluble']
print("\nگزارش معیارها برای هر کلاس:")
print(classification_report(y_balanced, y_pred, target_names=class_names))

# محاسبه MCC برای هر کلاس
print("\nMCC برای هر کلاس:")
for i, class_name in enumerate(class_names):
    y_true_binary = (y_balanced == i).astype(int)
    y_pred_binary = (y_pred == i).astype(int)
    mcc_class = matthews_corrcoef(y_true_binary, y_pred_binary)
    print(f"{class_name}: {mcc_class:.4f}")

Start processing CeFra-Seq Data phase
Number of gene ids: 6845
Unique gene ids: 6845
Number of all mRNA's: 6845
Length of y: 6845
چند شناسه ژن اول از CeFra-Seq: ['ENSG00000000003', 'ENSG00000000419', 'ENSG00000001084', 'ENSG00000001167', 'ENSG00000001460']
Cytosol: 2086 Nucleus: 2168 Membrane: 211 Insoluble: 2380
Shape of X DATA: (6845,)
Shape of y DATA: (6845,)
تعداد نمونه‌ها در هر کلاس (کل داده‌ها): [2086 2168  211 2380]
تعداد نمونه‌ها در TF-IDF: 6845
تعداد نمونه‌ها در Distance-based: 6845
شکل ماتریس PPI: (11373, 11373)
شکل ماتریس PPI بعد از فیلتر کردن: (6845, 6845)
شکل ویژگی‌های PPI بعد از PCA: (6845, 500)
شکل ماتریس ویژگی‌ها با PPI: (6845, 1668)
تعداد نمونه‌ها در هر کلاس بعد از SMOTE (با PPI): [2380 2380 2380 2380]
بهترین پارامترها: {'C': 100, 'gamma': 'scale'}
بهترین F1-score در Grid Search: 0.745510280451389

معیارهای کلی:
میانگین F1-score (ماکرو) با 10-fold cross-validation: 0.7512
Accuracy: 0.7550
MCC: 0.6742

گزارش معیارها برای هر کلاس:
              precision    recall  f1-sc

In [1]:
import os
import numpy as np
from Bio import SeqIO
from collections import Counter
from itertools import product
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import classification_report, matthews_corrcoef, accuracy_score, roc_auc_score, average_precision_score
from sklearn.feature_extraction.text import TfidfTransformer
from imblearn.over_sampling import SMOTE
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import joblib
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize
from torch.nn import functional as F

# تنظیم دستگاه (GPU یا CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"دستگاه: {device}")

# تابع برای Data Augmentation (تغییر تصادفی نوکلئوتیدها)
def augment_sequence(seq, mutation_rate=0.03):
    nucleotides = ['A', 'C', 'G', 'T']
    seq = list(seq)
    for i in range(len(seq)):
        if np.random.rand() < mutation_rate:
            seq[i] = np.random.choice([n for n in nucleotides if n != seq[i]])
    return ''.join(seq)

# تابع برای استخراج کلاس و مقادیر dist از هدر
def get_class_and_dist_from_header(header):
    class_names = ['Insoluble', 'Membrane', 'Nucleus', 'Cytosol']
    try:
        dist_part = header.split("dist:")[1].split()[0]
        dist_values = list(map(float, dist_part.split("_")))
        max_index = np.argmax(dist_values)
        return class_names[max_index], dist_values
    except (IndexError, ValueError):
        return None, None

# تابع برای استخراج ویژگی‌های TF (k-mer) و مقادیر dist
def extract_tf_features(fasta_files, k=5, augment=False):
    kmer_dict = {''.join(n): i for i, n in enumerate(product('ACGT', repeat=k))}
    features = []
    labels = []
    sequences = []
    dist_features = []
    class_to_label = {'Insoluble': 0, 'Membrane': 1, 'Nucleus': 2, 'Cytosol': 3}

    for fasta_file in fasta_files:
        for record in SeqIO.parse(fasta_file, "fasta"):
            seq = str(record.seq)
            header = record.description
            class_name, dist_values = get_class_and_dist_from_header(header)
            if class_name is None or dist_values is None:
                continue

            label = class_to_label[class_name]
            kmers = [seq[i:i+k] for i in range(len(seq)-k+1)]
            kmer_count = Counter(kmers)
            feature_vector = np.zeros(len(kmer_dict))
            for kmer, count in kmer_count.items():
                if kmer in kmer_dict:
                    feature_vector[kmer_dict[kmer]] = count / len(kmers)
            features.append(feature_vector)
            labels.append(label)
            sequences.append(seq)
            dist_features.append(dist_values)

            # اعمال Data Augmentation
            if augment:
                augmented_seq = augment_sequence(seq)
                kmers = [augmented_seq[i:i+k] for i in range(len(augmented_seq)-k+1)]
                kmer_count = Counter(kmers)
                feature_vector = np.zeros(len(kmer_dict))
                for kmer, count in kmer_count.items():
                    if kmer in kmer_dict:
                        feature_vector[kmer_dict[kmer]] = count / len(kmers)
                features.append(feature_vector)
                labels.append(label)
                sequences.append(augmented_seq)
                dist_features.append(dist_values)
    
    return np.array(features), np.array(labels), sequences, np.array(dist_features)

# تابع برای استخراج Distance-based Subsequence Profiles
def extract_distance_based_features(sequences, max_k=8):
    patterns = [''.join(p) for p in product('ACGT', repeat=2)]
    features = []
    
    for seq in sequences:
        feature_vector = np.zeros((max_k + 1) * len(patterns))
        for k in range(max_k + 1):
            for i, pattern in enumerate(patterns):
                count = 0
                for j in range(len(seq) - k - 2):
                    if seq[j] == pattern[0] and seq[j + k + 1] == pattern[1]:
                        count += 1
                feature_vector[k * len(patterns) + i] = count / (len(seq) - k - 1) if len(seq) - k - 1 > 0 else 0
        features.append(feature_vector)
    
    return np.array(features)

# تابع برای استخراج ویژگی‌های ساختار ثانویه (ساده‌شده)
def extract_ssf_features(sequences):
    features = []
    
    for seq in sequences:
        seq_len = len(seq)
        gc_content = (seq.count('G') + seq.count('C')) / seq_len if seq_len > 0 else 0
        au_pairs = sum(1 for i in range(seq_len-1) if (seq[i] == 'A' and seq[i+1] == 'U') or (seq[i] == 'U' and seq[i+1] == 'A'))
        gc_pairs = sum(1 for i in range(seq_len-1) if (seq[i] == 'G' and seq[i+1] == 'C') or (seq[i] == 'C' and seq[i+1] == 'G'))
        loop_count = sum(1 for i in range(seq_len-2) if seq[i:i+3] in ['AAA', 'UUU', 'AAU', 'UUA'])
        free_energy = -1.0 * gc_pairs - 0.5 * au_pairs
        
        feature_vector = np.array([
            gc_content,
            au_pairs / seq_len if seq_len > 0 else 0,
            gc_pairs / seq_len if seq_len > 0 else 0,
            loop_count / seq_len if seq_len > 0 else 0,
            free_energy / seq_len if seq_len > 0 else 0
        ])
        features.append(feature_vector)
    
    return np.array(features)

# تابع برای استخراج ویژگی‌های فیزیکوشیمیایی با کوتاه کردن توالی‌ها
def extract_pcp_features(sequences, max_length=1000):
    properties = {
        'A': [0.62, 0.3, -0.5],
        'C': [0.29, 0.4, -0.2],
        'G': [0.48, 0.5, -0.3],
        'T': [0.58, 0.2, -0.1],
        'U': [0.58, 0.2, -0.1],
        'N': [0.0, 0.0, 0.0]
    }
    
    features = []
    for seq in sequences:
        # کوتاه کردن توالی به max_length
        seq = seq[:max_length]
        prop_matrix = np.zeros((max_length, 3))
        for i in range(len(seq)):
            prop_matrix[i] = properties.get(seq[i], [0.0, 0.0, 0.0])
        mean_props = prop_matrix.mean(axis=0)
        std_props = prop_matrix.std(axis=0)
        feature_vector = np.concatenate([mean_props, std_props])
        features.append(feature_vector)
    
    return np.array(features)

# تعریف شاخه ویژگی (با Conv1D، BatchNorm و MultiHeadAttention)
class FeatureBranch(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(FeatureBranch, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=input_dim, out_channels=48, kernel_size=1, padding=0)
        self.bn = nn.BatchNorm1d(48)
        self.dropout1 = nn.Dropout(0.5)
        self.mha = nn.MultiheadAttention(embed_dim=48, num_heads=4)
        self.norm = nn.LayerNorm(48)
        self.linear = nn.Linear(48, output_dim)
    
    def forward(self, x):
        x = x.unsqueeze(2)
        x = self.conv1d(x)
        x = self.bn(x)
        x = self.dropout1(x)
        x = x.permute(2, 0, 1)
        x, _ = self.mha(x, x, x)
        x = self.norm(x)
        x = x.permute(1, 0, 2)
        x = x.squeeze(1)
        x = self.linear(x)
        return x

# تعریف مدل کامل با شاخه جدید برای dist
class CeFraSeqModel(nn.Module):
    def __init__(self, tfidf_dim=1024, distance_dim=144, ssf_dim=5, pcp_dim=6, dist_dim=4, linear_dim=256, num_classes=4):
        super(CeFraSeqModel, self).__init__()
        self.tfidf_branch = FeatureBranch(tfidf_dim, linear_dim)
        self.distance_branch = FeatureBranch(distance_dim, linear_dim)
        self.ssf_branch = FeatureBranch(ssf_dim, linear_dim)
        self.pcp_branch = FeatureBranch(pcp_dim, linear_dim)
        self.dist_branch = FeatureBranch(dist_dim, linear_dim)
        self.fc = nn.Linear(linear_dim * 5, 256)
        self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(256, num_classes)
    
    def forward(self, tfidf, distance, ssf, pcp, dist):
        tfidf_out = self.tfidf_branch(tfidf)
        distance_out = self.distance_branch(distance)
        ssf_out = self.ssf_branch(ssf)
        pcp_out = self.pcp_branch(pcp)
        dist_out = self.dist_branch(dist)
        combined = torch.cat((tfidf_out, distance_out, ssf_out, pcp_out, dist_out), dim=1)
        fc_out = F.relu(self.fc(combined))
        fc_out = self.dropout(fc_out)
        final_out = self.output(fc_out)
        return final_out, fc_out

# محاسبه وزن کلاس‌ها
def compute_class_weights(labels):
    class_counts = np.bincount(labels)
    n_classes = len(class_counts)
    n_samples = len(labels)
    weights = n_samples / (n_classes * class_counts)
    return torch.tensor(weights, dtype=torch.float).to(device)

# تابع آموزش مدل PyTorch
def train_model(model, train_loader, val_loader, num_epochs, device, checkpoint_path, class_weights, patience=7):
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    best_val_acc = 0.0
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for tfidf, distance, ssf, pcp, dist, labels in train_loader:
            tfidf, distance, ssf, pcp, dist, labels = tfidf.to(device), distance.to(device), ssf.to(device), pcp.to(device), dist.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs, _ = model(tfidf, distance, ssf, pcp, dist)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # اعتبارسنجی
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for tfidf, distance, ssf, pcp, dist, labels in val_loader:
                tfidf, distance, ssf, pcp, dist, labels = tfidf.to(device), distance.to(device), ssf.to(device), pcp.to(device), dist.to(device), labels.to(device)
                outputs, _ = model(tfidf, distance, ssf, pcp, dist)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_acc = val_correct / val_total
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Val Accuracy: {val_acc:.4f}")
        
        # ذخیره بهترین مدل
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"بهترین مدل ذخیره شد: {checkpoint_path}")
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"صبر: {patience_counter}/{patience}")
        
        # Early Stopping
        if patience_counter >= patience:
            print("Early Stopping: توقف آموزش به دلیل عدم بهبود در دقت اعتبارسنجی")
            break

# تابع برای استخراج ویژگی‌ها از مدل PyTorch
def extract_features(model, data_loader, device):
    model.eval()
    features = []
    labels_list = []
    
    with torch.no_grad():
        for tfidf, distance, ssf, pcp, dist, labels in data_loader:
            tfidf, distance, ssf, pcp, dist, labels = tfidf.to(device), distance.to(device), ssf.to(device), pcp.to(device), dist.to(device), labels.to(device)
            _, fc_out = model(tfidf, distance, ssf, pcp, dist)
            features.append(fc_out.cpu().numpy())
            labels_list.append(labels.cpu().numpy())
    
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels_list, axis=0)
    return features, labels

# فایل‌های FASTA
fasta_files = [
    "F:\\New Version\\Data\\cefra-seq\\cefra_seq_cDNA_ann_screened.fa",
    "F:\\New Version\\Data\\cefra-seq\\cefra_seq_cDNA_screened.fa"
]

# استخراج ویژگی‌های TF (با Data Augmentation) و مقادیر dist
tf_features, y, sequences, dist_features = extract_tf_features(fasta_files, k=5, augment=True)

# چاپ تعداد نمونه‌ها در هر کلاس
print("تعداد نمونه‌ها در هر کلاس (کل داده‌ها):", np.bincount(y))

# تبدیل TF به TF-IDF
tfidf = TfidfTransformer()
tfidf_features = tfidf.fit_transform(tf_features).toarray()

# استخراج Distance-based Subsequence Profiles
distance_features = extract_distance_based_features(sequences, max_k=8)

# استخراج Secondary Structure Features
ssf_features = extract_ssf_features(sequences)

# استخراج Physicochemical Properties
pcp_features = extract_pcp_features(sequences, max_length=1000)

# بررسی تعداد نمونه‌ها
print("تعداد نمونه‌ها در TF-IDF:", tfidf_features.shape[0])
print("تعداد نمونه‌ها در Distance-based:", distance_features.shape[0])
print("تعداد نمونه‌ها در SSF:", ssf_features.shape[0])
print("تعداد نمونه‌ها در PCP:", pcp_features.shape[0])
print("تعداد نمونه‌ها در Dist:", dist_features.shape[0])

# نرمال‌سازی ویژگی‌ها
scaler_tfidf = StandardScaler()
scaler_distance = StandardScaler()
scaler_ssf = StandardScaler()
scaler_pcp = StandardScaler()
scaler_dist = StandardScaler()
tfidf_features_scaled = scaler_tfidf.fit_transform(tfidf_features)
distance_features_scaled = scaler_distance.fit_transform(distance_features)
ssf_features_scaled = scaler_ssf.fit_transform(ssf_features)
pcp_features_scaled = scaler_pcp.fit_transform(pcp_features)
dist_features_scaled = scaler_dist.fit_transform(dist_features)

# تقسیم داده‌ها به train و test (80% train، 20% test)
X_tfidf_train, X_tfidf_test, X_distance_train, X_distance_test, X_ssf_train, X_ssf_test, X_pcp_train, X_pcp_test, X_dist_train, X_dist_test, y_train, y_test = train_test_split(
    tfidf_features_scaled, distance_features_scaled, ssf_features_scaled, pcp_features_scaled, dist_features_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# چاپ تعداد نمونه‌ها در train و test
print("تعداد نمونه‌ها در train:", X_tfidf_train.shape[0])
print("تعداد نمونه‌ها در test:", X_tfidf_test.shape[0])
print("تعداد نمونه‌ها در هر کلاس (train):", np.bincount(y_train))
print("تعداد نمونه‌ها در هر کلاس (test):", np.bincount(y_test))

# اعمال SMOTE با استراتژی اصلاح‌شده
sampling_strategy = {0: 11000, 1: 11500, 2: 5000, 3: 11500}  # تنظیم برای تعادل کلاس‌ها
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
X_tfidf_train_balanced, y_train_balanced = smote.fit_resample(X_tfidf_train, y_train)
X_distance_train_balanced, _ = smote.fit_resample(X_distance_train, y_train)
X_ssf_train_balanced, _ = smote.fit_resample(X_ssf_train, y_train)
X_pcp_train_balanced, _ = smote.fit_resample(X_pcp_train, y_train)
X_dist_train_balanced, _ = smote.fit_resample(X_dist_train, y_train)

# چاپ تعداد نمونه‌ها بعد از SMOTE
print("تعداد نمونه‌ها در هر کلاس بعد از SMOTE (train):", np.bincount(y_train_balanced))

# تبدیل داده‌ها به Tensor برای PyTorch
train_dataset = TensorDataset(
    torch.tensor(X_tfidf_train_balanced, dtype=torch.float32),
    torch.tensor(X_distance_train_balanced, dtype=torch.float32),
    torch.tensor(X_ssf_train_balanced, dtype=torch.float32),
    torch.tensor(X_pcp_train_balanced, dtype=torch.float32),
    torch.tensor(X_dist_train_balanced, dtype=torch.float32),
    torch.tensor(y_train_balanced, dtype=torch.long)
)
test_dataset = TensorDataset(
    torch.tensor(X_tfidf_test, dtype=torch.float32),
    torch.tensor(X_distance_test, dtype=torch.float32),
    torch.tensor(X_ssf_test, dtype=torch.float32),
    torch.tensor(X_pcp_test, dtype=torch.float32),
    torch.tensor(X_dist_test, dtype=torch.float32),
    torch.tensor(y_test, dtype=torch.long)
)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

# محاسبه وزن کلاس‌ها برای PyTorch
class_weights = compute_class_weights(y_train_balanced)

# آموزش مدل PyTorch
model = CeFraSeqModel(tfidf_dim=1024, distance_dim=144, ssf_dim=5, pcp_dim=6, dist_dim=4, linear_dim=256, num_classes=4).to(device)
train_model(model, train_loader, test_loader, num_epochs=40, device=device, checkpoint_path="best_cefra_model.pth", class_weights=class_weights, patience=7)

# بارگذاری بهترین مدل PyTorch
model.load_state_dict(torch.load("best_cefra_model.pth"))
print("بهترین مدل PyTorch بارگذاری شد.")

# استخراج ویژگی‌ها از مدل PyTorch
X_train_features, y_train_features = extract_features(model, train_loader, device)
X_test_features, y_test_features = extract_features(model, test_loader, device)

# بهینه‌سازی SVM با Grid Search روی ویژگی‌های استخراج‌شده
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale', 'auto']
}
svm_model = SVC(probability=True, random_state=42)
grid = GridSearchCV(svm_model, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid.fit(X_train_features, y_train_features)

# چاپ بهترین پارامترها
print("بهترین پارامترها:", grid.best_params_)
print("بهترین F1-score در Grid Search:", grid.best_score_)

# استفاده از بهترین مدل SVM
best_svm = grid.best_estimator_

# ارزیابی مدل روی داده آموزش با cross-validation
y_train_pred = cross_val_predict(best_svm, X_train_features, y_train_features, cv=10)
f1_macro_train = cross_val_score(best_svm, X_train_features, y_train_features, cv=10, scoring='f1_macro').mean()
accuracy_train = accuracy_score(y_train_features, y_train_pred)
mcc_train = matthews_corrcoef(y_train_features, y_train_pred)

print("\nمعیارهای کلی روی داده آموزش (cross-validation):")
print(f"میانگین F1-score (ماکرو) با 10-fold cross-validation: {f1_macro_train:.4f}")
print(f"Accuracy: {accuracy_train:.4f}")
print(f"MCC: {mcc_train:.4f}")

# گزارش معیارها برای هر کلاس روی داده آموزش
class_names = ['Insoluble', 'Membrane', 'Nucleus', 'Cytosol']
print("\nگزارش معیارها برای هر کلاس (train):")
print(classification_report(y_train_features, y_train_pred, target_names=class_names))

# ارزیابی مدل روی داده تست
y_test_pred = best_svm.predict(X_test_features)
y_test_pred_proba = best_svm.predict_proba(X_test_features)
f1_macro_test = cross_val_score(best_svm, X_test_features, y_test_features, cv=10, scoring='f1_macro').mean()
accuracy_test = accuracy_score(y_test_features, y_test_pred)
mcc_test = matthews_corrcoef(y_test_features, y_test_pred)

# محاسبه AUC-ROC و AUC-PR
y_test_bin = label_binarize(y_test_features, classes=[0, 1, 2, 3])
auc_roc_test = roc_auc_score(y_test_bin, y_test_pred_proba, average='macro')
auc_pr_test = average_precision_score(y_test_bin, y_test_pred_proba, average='macro')

print("\nمعیارهای کلی روی داده تست:")
print(f"F1-score (ماکرو): {f1_macro_test:.4f}")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"MCC: {mcc_test:.4f}")
print(f"AUC-ROC: {auc_roc_test:.4f}")
print(f"AUC-PR: {auc_pr_test:.4f}")

# گزارش معیارها برای هر کلاس روی داده تست
print("\nگزارش معیارها برای هر کلاس (test):")
print(classification_report(y_test_features, y_test_pred, target_names=class_names))

# محاسبه MCC برای هر کلاس روی داده تست
print("\nMCC برای هر کلاس (test):")
for i, class_name in enumerate(class_names):
    y_true_binary = (y_test_features == i).astype(int)
    y_pred_binary = (y_test_pred == i).astype(int)
    mcc_class = matthews_corrcoef(y_true_binary, y_pred_binary)
    print(f"{class_name}: {mcc_class:.4f}")

# ذخیره مدل برای استفاده بعدی
joblib.dump(best_svm, 'best_cefra_svm_model.pkl')
print("\nمدل ذخیره شد: best_cefra_svm_model.pkl")

دستگاه: cuda
تعداد نمونه‌ها در هر کلاس (کل داده‌ها): [13408 13836  4320 13928]
تعداد نمونه‌ها در TF-IDF: 45492
تعداد نمونه‌ها در Distance-based: 45492
تعداد نمونه‌ها در SSF: 45492
تعداد نمونه‌ها در PCP: 45492
تعداد نمونه‌ها در Dist: 45492
تعداد نمونه‌ها در train: 36393
تعداد نمونه‌ها در test: 9099
تعداد نمونه‌ها در هر کلاس (train): [10726 11069  3456 11142]
تعداد نمونه‌ها در هر کلاس (test): [2682 2767  864 2786]
تعداد نمونه‌ها در هر کلاس بعد از SMOTE (train): [11000 11500  5000 11500]
Epoch 1/40, Train Loss: 1.1466, Val Accuracy: 0.4726
بهترین مدل ذخیره شد: best_cefra_model.pth
Epoch 2/40, Train Loss: 0.9987, Val Accuracy: 0.4335
صبر: 1/7
Epoch 3/40, Train Loss: 0.9229, Val Accuracy: 0.5472
بهترین مدل ذخیره شد: best_cefra_model.pth
Epoch 4/40, Train Loss: 0.9015, Val Accuracy: 0.4236
صبر: 1/7
Epoch 5/40, Train Loss: 0.8673, Val Accuracy: 0.4958
صبر: 2/7
Epoch 6/40, Train Loss: 0.8723, Val Accuracy: 0.3736
صبر: 3/7
Epoch 7/40, Train Loss: 0.8474, Val Accuracy: 0.4099
صبر: 4/7
Epoch 8/40

In [ ]:
#=========cefra_seq_model_fainaly=====================
import os
import numpy as np
from Bio import SeqIO
from collections import Counter
from itertools import product
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import classification_report, matthews_corrcoef, accuracy_score, roc_auc_score, average_precision_score, f1_score
from sklearn.feature_extraction.text import TfidfTransformer
from imblearn.over_sampling import SMOTE
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import joblib
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize
from torch.nn import functional as F
from scipy.stats import pearsonr

# تنظیم دستگاه (GPU یا CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"دستگاه: {device}")

# تابع برای Data Augmentation (تغییر تصادفی نوکلئوتیدها)
def augment_sequence(seq, mutation_rate=0.03):
    nucleotides = ['A', 'C', 'G', 'T']
    seq = list(seq)
    for i in range(len(seq)):
        if np.random.rand() < mutation_rate:
            seq[i] = np.random.choice([n for n in nucleotides if n != seq[i]])
    return ''.join(seq)

# تابع برای استخراج کلاس و مقادیر dist از هدر
def get_class_and_dist_from_header(header):
    class_names = ['Insoluble', 'Membrane', 'Nucleus', 'Cytosol']
    try:
        dist_part = header.split("dist:")[1].split()[0]
        dist_values = list(map(float, dist_part.split("_")))
        max_index = np.argmax(dist_values)
        return class_names[max_index], dist_values
    except (IndexError, ValueError):
        return None, None

# تابع برای استخراج ویژگی‌های TF (k-mer) و مقادیر dist
def extract_tf_features(fasta_files, k=5, augment=False):
    kmer_dict = {''.join(n): i for i, n in enumerate(product('ACGT', repeat=k))}
    features = []
    labels = []
    sequences = []
    dist_features = []
    class_to_label = {'Insoluble': 0, 'Membrane': 1, 'Nucleus': 2, 'Cytosol': 3}

    for fasta_file in fasta_files:
        for record in SeqIO.parse(fasta_file, "fasta"):
            seq = str(record.seq)
            header = record.description
            class_name, dist_values = get_class_and_dist_from_header(header)
            if class_name is None or dist_values is None:
                continue

            label = class_to_label[class_name]
            kmers = [seq[i:i+k] for i in range(len(seq)-k+1)]
            kmer_count = Counter(kmers)
            feature_vector = np.zeros(len(kmer_dict))
            for kmer, count in kmer_count.items():
                if kmer in kmer_dict:
                    feature_vector[kmer_dict[kmer]] = count / len(kmers)
            features.append(feature_vector)
            labels.append(label)
            sequences.append(seq)
            dist_features.append(dist_values)

            # اعمال Data Augmentation
            if augment:
                augmented_seq = augment_sequence(seq)
                kmers = [augmented_seq[i:i+k] for i in range(len(augmented_seq)-k+1)]
                kmer_count = Counter(kmers)
                feature_vector = np.zeros(len(kmer_dict))
                for kmer, count in kmer_count.items():
                    if kmer in kmer_dict:
                        feature_vector[kmer_dict[kmer]] = count / len(kmers)
                features.append(feature_vector)
                labels.append(label)
                sequences.append(augmented_seq)
                dist_features.append(dist_values)
    
    return np.array(features), np.array(labels), sequences, np.array(dist_features)

# تابع برای استخراج Distance-based Subsequence Profiles
def extract_distance_based_features(sequences, max_k=8):
    patterns = [''.join(p) for p in product('ACGT', repeat=2)]
    features = []
    
    for seq in sequences:
        feature_vector = np.zeros((max_k + 1) * len(patterns))
        for k in range(max_k + 1):
            for i, pattern in enumerate(patterns):
                count = 0
                for j in range(len(seq) - k - 2):
                    if seq[j] == pattern[0] and seq[j + k + 1] == pattern[1]:
                        count += 1
                feature_vector[k * len(patterns) + i] = count / (len(seq) - k - 1) if len(seq) - k - 1 > 0 else 0
        features.append(feature_vector)
    
    return np.array(features)

# تابع برای استخراج ویژگی‌های ساختار ثانویه (ساده‌شده)
def extract_ssf_features(sequences):
    features = []
    
    for seq in sequences:
        seq_len = len(seq)
        gc_content = (seq.count('G') + seq.count('C')) / seq_len if seq_len > 0 else 0
        au_pairs = sum(1 for i in range(seq_len-1) if (seq[i] == 'A' and seq[i+1] == 'U') or (seq[i] == 'U' and seq[i+1] == 'A'))
        gc_pairs = sum(1 for i in range(seq_len-1) if (seq[i] == 'G' and seq[i+1] == 'C') or (seq[i] == 'C' and seq[i+1] == 'G'))
        loop_count = sum(1 for i in range(seq_len-2) if seq[i:i+3] in ['AAA', 'UUU', 'AAU', 'UUA'])
        free_energy = -1.0 * gc_pairs - 0.5 * au_pairs
        
        feature_vector = np.array([
            gc_content,
            au_pairs / seq_len if seq_len > 0 else 0,
            gc_pairs / seq_len if seq_len > 0 else 0,
            loop_count / seq_len if seq_len > 0 else 0,
            free_energy / seq_len if seq_len > 0 else 0
        ])
        features.append(feature_vector)
    
    return np.array(features)

# تابع برای استخراج ویژگی‌های فیزیکوشیمیایی با کوتاه کردن توالی‌ها
def extract_pcp_features(sequences, max_length=1000):
    properties = {
        'A': [0.62, 0.3, -0.5],
        'C': [0.29, 0.4, -0.2],
        'G': [0.48, 0.5, -0.3],
        'T': [0.58, 0.2, -0.1],
        'U': [0.58, 0.2, -0.1],
        'N': [0.0, 0.0, 0.0]
    }
    
    features = []
    for seq in sequences:
        # کوتاه کردن توالی به max_length
        seq = seq[:max_length]
        prop_matrix = np.zeros((max_length, 3))
        for i in range(len(seq)):
            prop_matrix[i] = properties.get(seq[i], [0.0, 0.0, 0.0])
        mean_props = prop_matrix.mean(axis=0)
        std_props = prop_matrix.std(axis=0)
        feature_vector = np.concatenate([mean_props, std_props])
        features.append(feature_vector)
    
    return np.array(features)

# تعریف شاخه ویژگی (با Conv1D، BatchNorm و MultiHeadAttention)
class FeatureBranch(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(FeatureBranch, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=input_dim, out_channels=48, kernel_size=1, padding=0)
        self.bn = nn.BatchNorm1d(48)
        self.dropout1 = nn.Dropout(0.5)
        self.mha = nn.MultiheadAttention(embed_dim=48, num_heads=4)
        self.norm = nn.LayerNorm(48)
        self.linear = nn.Linear(48, output_dim)
    
    def forward(self, x):
        x = x.unsqueeze(2)
        x = self.conv1d(x)
        x = self.bn(x)
        x = self.dropout1(x)
        x = x.permute(2, 0, 1)
        x, _ = self.mha(x, x, x)
        x = self.norm(x)
        x = x.permute(1, 0, 2)
        x = x.squeeze(1)
        x = self.linear(x)
        return x

# تعریف مدل کامل با شاخه جدید برای dist
class CeFraSeqModel(nn.Module):
    def __init__(self, tfidf_dim=1024, distance_dim=144, ssf_dim=5, pcp_dim=6, dist_dim=4, linear_dim=256, num_classes=4):
        super(CeFraSeqModel, self).__init__()
        self.tfidf_branch = FeatureBranch(tfidf_dim, linear_dim)
        self.distance_branch = FeatureBranch(distance_dim, linear_dim)
        self.ssf_branch = FeatureBranch(ssf_dim, linear_dim)
        self.pcp_branch = FeatureBranch(pcp_dim, linear_dim)
        self.dist_branch = FeatureBranch(dist_dim, linear_dim)
        self.fc = nn.Linear(linear_dim * 5, 256)
        self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(256, num_classes)
    
    def forward(self, tfidf, distance, ssf, pcp, dist):
        tfidf_out = self.tfidf_branch(tfidf)
        distance_out = self.distance_branch(distance)
        ssf_out = self.ssf_branch(ssf)
        pcp_out = self.pcp_branch(pcp)
        dist_out = self.dist_branch(dist)
        combined = torch.cat((tfidf_out, distance_out, ssf_out, pcp_out, dist_out), dim=1)
        fc_out = F.relu(self.fc(combined))
        fc_out = self.dropout(fc_out)
        final_out = self.output(fc_out)
        return final_out, fc_out

# محاسبه وزن کلاس‌ها
def compute_class_weights(labels):
    class_counts = np.bincount(labels)
    n_classes = len(class_counts)
    n_samples = len(labels)
    weights = n_samples / (n_classes * class_counts)
    return torch.tensor(weights, dtype=torch.float).to(device)

# تابع آموزش مدل PyTorch
def train_model(model, train_loader, val_loader, num_epochs, device, checkpoint_path, class_weights, patience=7):
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    best_val_acc = 0.0
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for tfidf, distance, ssf, pcp, dist, labels in train_loader:
            tfidf, distance, ssf, pcp, dist, labels = tfidf.to(device), distance.to(device), ssf.to(device), pcp.to(device), dist.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs, _ = model(tfidf, distance, ssf, pcp, dist)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # اعتبارسنجی
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for tfidf, distance, ssf, pcp, dist, labels in val_loader:
                tfidf, distance, ssf, pcp, dist, labels = tfidf.to(device), distance.to(device), ssf.to(device), pcp.to(device), dist.to(device), labels.to(device)
                outputs, _ = model(tfidf, distance, ssf, pcp, dist)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_acc = val_correct / val_total
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Val Accuracy: {val_acc:.4f}")
        
        # ذخیره بهترین مدل
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"بهترین مدل ذخیره شد: {checkpoint_path}")
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"صبر: {patience_counter}/{patience}")
        
        # Early Stopping
        if patience_counter >= patience:
            print("Early Stopping: توقف آموزش به دلیل عدم بهبود در دقت اعتبارسنجی")
            break

# تابع برای استخراج ویژگی‌ها از مدل PyTorch
def extract_features(model, data_loader, device):
    model.eval()
    features = []
    labels_list = []
    
    with torch.no_grad():
        for tfidf, distance, ssf, pcp, dist, labels in data_loader:
            tfidf, distance, ssf, pcp, dist, labels = tfidf.to(device), distance.to(device), ssf.to(device), pcp.to(device), dist.to(device), labels.to(device)
            _, fc_out = model(tfidf, distance, ssf, pcp, dist)
            features.append(fc_out.cpu().numpy())
            labels_list.append(labels.cpu().numpy())
    
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels_list, axis=0)
    return features, labels

# مسیر ذخیره‌سازی
output_dir = "F:\\payan-nameh\\Cefra-seq"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# فایل‌های FASTA
fasta_files = [
    "F:\\New Version\\Data\\cefra-seq\\cefra_seq_cDNA_ann_screened.fa",
    "F:\\New Version\\Data\\cefra-seq\\cefra_seq_cDNA_screened.fa"
]

# استخراج ویژگی‌های TF (با Data Augmentation) و مقادیر dist
tf_features, y, sequences, dist_features = extract_tf_features(fasta_files, k=5, augment=True)

# چاپ تعداد نمونه‌ها در هر کلاس
print("تعداد نمونه‌ها در هر کلاس (کل داده‌ها):", np.bincount(y))

# تبدیل TF به TF-IDF
tfidf = TfidfTransformer()
tfidf_features = tfidf.fit_transform(tf_features).toarray()

# استخراج Distance-based Subsequence Profiles
distance_features = extract_distance_based_features(sequences, max_k=8)

# استخراج Secondary Structure Features
ssf_features = extract_ssf_features(sequences)

# استخراج Physicochemical Properties
pcp_features = extract_pcp_features(sequences, max_length=1000)

# بررسی تعداد نمونه‌ها
print("تعداد نمونه‌ها در TF-IDF:", tfidf_features.shape[0])
print("تعداد نمونه‌ها در Distance-based:", distance_features.shape[0])
print("تعداد نمونه‌ها در SSF:", ssf_features.shape[0])
print("تعداد نمونه‌ها در PCP:", pcp_features.shape[0])
print("تعداد نمونه‌ها در Dist:", dist_features.shape[0])

# نرمال‌سازی ویژگی‌ها
scaler_tfidf = StandardScaler()
scaler_distance = StandardScaler()
scaler_ssf = StandardScaler()
scaler_pcp = StandardScaler()
scaler_dist = StandardScaler()
tfidf_features_scaled = scaler_tfidf.fit_transform(tfidf_features)
distance_features_scaled = scaler_distance.fit_transform(distance_features)
ssf_features_scaled = scaler_ssf.fit_transform(ssf_features)
pcp_features_scaled = scaler_pcp.fit_transform(pcp_features)
dist_features_scaled = scaler_dist.fit_transform(dist_features)

# ذخیره اسکیلرها
joblib.dump(scaler_tfidf, os.path.join(output_dir, 'scaler_tfidf.pkl'))
joblib.dump(scaler_distance, os.path.join(output_dir, 'scaler_distance.pkl'))
joblib.dump(scaler_ssf, os.path.join(output_dir, 'scaler_ssf.pkl'))
joblib.dump(scaler_pcp, os.path.join(output_dir, 'scaler_pcp.pkl'))
joblib.dump(scaler_dist, os.path.join(output_dir, 'scaler_dist.pkl'))
print("اسکیلرها ذخیره شدند.")

# تقسیم داده‌ها به train و test (80% train، 20% test)
X_tfidf_train, X_tfidf_test, X_distance_train, X_distance_test, X_ssf_train, X_ssf_test, X_pcp_train, X_pcp_test, X_dist_train, X_dist_test, y_train, y_test = train_test_split(
    tfidf_features_scaled, distance_features_scaled, ssf_features_scaled, pcp_features_scaled, dist_features_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# چاپ تعداد نمونه‌ها در train و test
print("تعداد نمونه‌ها در train:", X_tfidf_train.shape[0])
print("تعداد نمونه‌ها در test:", X_tfidf_test.shape[0])
print("تعداد نمونه‌ها در هر کلاس (train):", np.bincount(y_train))
print("تعداد نمونه‌ها در هر کلاس (test):", np.bincount(y_test))

# اعمال SMOTE با استراتژی اصلاح‌شده
sampling_strategy = {0: 11000, 1: 11500, 2: 5000, 3: 11500}  # تنظیم برای تعادل کلاس‌ها
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
X_tfidf_train_balanced, y_train_balanced = smote.fit_resample(X_tfidf_train, y_train)
X_distance_train_balanced, _ = smote.fit_resample(X_distance_train, y_train)
X_ssf_train_balanced, _ = smote.fit_resample(X_ssf_train, y_train)
X_pcp_train_balanced, _ = smote.fit_resample(X_pcp_train, y_train)
X_dist_train_balanced, _ = smote.fit_resample(X_dist_train, y_train)

# چاپ تعداد نمونه‌ها بعد از SMOTE
print("تعداد نمونه‌ها در هر کلاس بعد از SMOTE (train):", np.bincount(y_train_balanced))

# تبدیل داده‌ها به Tensor برای PyTorch
train_dataset = TensorDataset(
    torch.tensor(X_tfidf_train_balanced, dtype=torch.float32),
    torch.tensor(X_distance_train_balanced, dtype=torch.float32),
    torch.tensor(X_ssf_train_balanced, dtype=torch.float32),
    torch.tensor(X_pcp_train_balanced, dtype=torch.float32),
    torch.tensor(X_dist_train_balanced, dtype=torch.float32),
    torch.tensor(y_train_balanced, dtype=torch.long)
)
test_dataset = TensorDataset(
    torch.tensor(X_tfidf_test, dtype=torch.float32),
    torch.tensor(X_distance_test, dtype=torch.float32),
    torch.tensor(X_ssf_test, dtype=torch.float32),
    torch.tensor(X_pcp_test, dtype=torch.float32),
    torch.tensor(X_dist_test, dtype=torch.float32),
    torch.tensor(y_test, dtype=torch.long)
)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

# محاسبه وزن کلاس‌ها برای PyTorch
class_weights = compute_class_weights(y_train_balanced)

# آموزش مدل PyTorch
model = CeFraSeqModel(tfidf_dim=1024, distance_dim=144, ssf_dim=5, pcp_dim=6, dist_dim=4, linear_dim=256, num_classes=4).to(device)
train_model(model, train_loader, test_loader, num_epochs=40, device=device, checkpoint_path=os.path.join(output_dir, "best_cefra_model.pth"), class_weights=class_weights, patience=7)

# بارگذاری بهترین مدل PyTorch
model.load_state_dict(torch.load(os.path.join(output_dir, "best_cefra_model.pth")))
print("بهترین مدل PyTorch بارگذاری شد.")

# استخراج ویژگی‌ها از مدل PyTorch
X_train_features, y_train_features = extract_features(model, train_loader, device)
X_test_features, y_test_features = extract_features(model, test_loader, device)

# ذخیره ویژگی‌ها
np.save(os.path.join(output_dir, 'X_test_features.npy'), X_test_features)
np.save(os.path.join(output_dir, 'y_test_features.npy'), y_test_features)
print("ویژگی‌های تست ذخیره شدند.")

# بهینه‌سازی SVM با Grid Search روی ویژگی‌های استخراج‌شده
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale', 'auto']
}
svm_model = SVC(probability=True, random_state=42)
grid = GridSearchCV(svm_model, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid.fit(X_train_features, y_train_features)

# چاپ بهترین پارامترها
print("بهترین پارامترها:", grid.best_params_)
print("بهترین F1-score در Grid Search:", grid.best_score_)

# استفاده از بهترین مدل SVM
best_svm = grid.best_estimator_

# ارزیابی مدل روی داده آموزش با cross-validation
y_train_pred = cross_val_predict(best_svm, X_train_features, y_train_features, cv=10)
f1_macro_train = cross_val_score(best_svm, X_train_features, y_train_features, cv=10, scoring='f1_macro').mean()
accuracy_train = accuracy_score(y_train_features, y_train_pred)
mcc_train = matthews_corrcoef(y_train_features, y_train_pred)

print("\nمعیارهای کلی روی داده آموزش (cross-validation):")
print(f"میانگین F1-score (ماکرو) با 10-fold cross-validation: {f1_macro_train:.4f}")
print(f"Accuracy: {accuracy_train:.4f}")
print(f"MCC: {mcc_train:.4f}")

# گزارش معیارها برای هر کلاس روی داده آموزش
class_names = ['Insoluble', 'Membrane', 'Nucleus', 'Cytosol']
print("\nگزارش معیارها برای هر کلاس (train):")
print(classification_report(y_train_features, y_train_pred, target_names=class_names))

# ارزیابی مدل روی داده تست
y_test_pred = best_svm.predict(X_test_features)
y_test_pred_proba = best_svm.predict_proba(X_test_features)
f1_macro_test = cross_val_score(best_svm, X_test_features, y_test_features, cv=10, scoring='f1_macro').mean()
accuracy_test = accuracy_score(y_test_features, y_test_pred)
mcc_test = matthews_corrcoef(y_test_features, y_test_pred)

# محاسبه AUC-ROC و AUC-PR
y_test_bin = label_binarize(y_test_features, classes=[0, 1, 2, 3])
auc_roc_test = roc_auc_score(y_test_bin, y_test_pred_proba, average='macro')
auc_pr_test = average_precision_score(y_test_bin, y_test_pred_proba, average='macro')

# محاسبه PCC برای هر کلاس و میانگین کلی
pcc_per_class = []
pcc_macro = 0.0
for i, class_name in enumerate(class_names):
    y_true_binary = (y_test_features == i).astype(int)
    y_pred_binary = (y_test_pred == i).astype(int)
    pcc, _ = pearsonr(y_true_binary, y_pred_binary)
    pcc_per_class.append(pcc)
    print(f"PCC برای {class_name} (test): {pcc:.4f}")
pcc_macro = np.mean(pcc_per_class) if pcc_per_class else 0.0
print(f"PCC (ماکرو) برای کل داده تست: {pcc_macro:.4f}")

print("\nمعیارهای کلی روی داده تست:")
print(f"F1-score (ماکرو): {f1_macro_test:.4f}")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"MCC: {mcc_test:.4f}")
print(f"AUC-ROC: {auc_roc_test:.4f}")
print(f"AUC-PR: {auc_pr_test:.4f}")
print(f"PCC (ماکرو): {pcc_macro:.4f}")

# گزارش معیارها برای هر کلاس روی داده تست
print("\nگزارش معیارها برای هر کلاس (test):")
print(classification_report(y_test_features, y_test_pred, target_names=class_names))

# محاسبه MCC برای هر کلاس روی داده تست
print("\nMCC برای هر کلاس (test):")
for i, class_name in enumerate(class_names):
    y_true_binary = (y_test_features == i).astype(int)
    y_pred_binary = (y_test_pred == i).astype(int)
    mcc_class = matthews_corrcoef(y_true_binary, y_pred_binary)
    print(f"{class_name}: {mcc_class:.4f}")

# ذخیره مدل برای استفاده بعدی
joblib.dump(best_svm, os.path.join(output_dir, 'best_cefra_svm_model.pkl'))
print(f"\nمدل ذخیره شد: {os.path.join(output_dir, 'best_cefra_svm_model.pkl')}")

In [ ]:
#===================lot_cefra_evaluation_charts.=============================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import joblib
from sklearn.preprocessing import label_binarize
from scipy.stats import pearsonr

# تنظیم مسیر ذخیره‌سازی
output_dir = "F:\\payan-nameh\\Cefra-seq"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# بارگذاری داده‌های ذخیره‌شده
X_test_features = np.load(os.path.join(output_dir, 'X_test_features.npy'))
y_test_features = np.load(os.path.join(output_dir, 'y_test_features.npy'))

# بارگذاری مدل SVM
best_svm = joblib.load(os.path.join(output_dir, 'best_cefra_svm_model.pkl'))
print(f"مدل SVM از مسیر {os.path.join(output_dir, 'best_cefra_svm_model.pkl')} بارگذاری شد.")

# پیش‌بینی روی داده‌های تست
y_test_pred = best_svm.predict(X_test_features)
y_test_pred_proba = best_svm.predict_proba(X_test_features)

# تعریف نام کلاس‌ها
class_names = ['Insoluble', 'Membrane', 'Nucleus', 'Cytosol']

# 1. رسم نمودار Loss و Accuracy برای داده‌های آموزش
# (فرض می‌کنیم train_loss و train_accuracy از مدل PyTorch ذخیره شده)
train_losses = []  # باید از خروجی train_model پر شود (در اینجا فرضی است)
train_accuracies = []  # باید از خروجی train_model پر شود (در اینجا فرضی است)
val_accuracies = []  # باید از خروجی train_model پر شود (در اینجا فرضی است)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss', color='red')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Train Accuracy', color='blue')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy', color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy over Epochs')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'train_loss_accuracy.png'))
plt.close()
print(f"نمودار Loss و Accuracy ذخیره شد: {os.path.join(output_dir, 'train_loss_accuracy.png')}")

# 2. رسم منحنی ROC برای هر کلاس (داده‌های تست)
y_test_bin = label_binarize(y_test_features, classes=[0, 1, 2, 3])
plt.figure(figsize=(10, 8))
for i in range(len(class_names)):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_test_pred_proba[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Each Class')
plt.legend(loc="lower right")
plt.savefig(os.path.join(output_dir, 'roc_curve.png'))
plt.close()
print(f"منحنی ROC ذخیره شد: {os.path.join(output_dir, 'roc_curve.png')}")

# 3. رسم منحنی Precision-Recall برای هر کلاس (داده‌های تست)
plt.figure(figsize=(10, 8))
for i in range(len(class_names)):
    precision, recall, _ = precision_recall_curve(y_test_bin[:, i], y_test_pred_proba[:, i])
    plt.plot(recall, precision, label=f'{class_names[i]}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for Each Class')
plt.legend(loc="lower left")
plt.savefig(os.path.join(output_dir, 'precision_recall_curve.png'))
plt.close()
print(f"منحنی Precision-Recall ذخیره شد: {os.path.join(output_dir, 'precision_recall_curve.png')}")

# 4. رسم نمودار میله‌ای برای F1، MCC، PCC (داده‌های تست)
f1_per_class = []
mcc_per_class = []
pcc_per_class = []
for i in range(len(class_names)):
    y_true_binary = (y_test_features == i).astype(int)
    y_pred_binary = (y_test_pred == i).astype(int)
    f1_per_class.append(f1_score(y_true_binary, y_pred_binary))
    mcc_per_class.append(matthews_corrcoef(y_true_binary, y_pred_binary))
    pcc, _ = pearsonr(y_true_binary, y_pred_binary)
    pcc_per_class.append(pcc)

metrics_df = pd.DataFrame({
    'Class': class_names,
    'F1': f1_per_class,
    'MCC': mcc_per_class,
    'PCC': pcc_per_class
})

plt.figure(figsize=(12, 6))
metrics_df.set_index('Class')[['F1', 'MCC', 'PCC']].plot(kind='bar', figsize=(12, 6))
plt.title('F1, MCC, and PCC per Class (Test Data)')
plt.ylabel('Score')
plt.xlabel('Class')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'f1_mcc_pcc_per_class.png'))
plt.close()
print(f"نمودار F1، MCC و PCC ذخیره شد: {os.path.join(output_dir, 'f1_mcc_pcc_per_class.png')}")

In [ ]:
#======================analyze_cefra_predictions_per_class===================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from Bio import SeqIO
from sklearn.model_selection import train_test_split

# تنظیم مسیر ذخیره‌سازی
output_dir = "F:\\payan-nameh\\Cefra-seq"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# تعریف نام کلاس‌ها
class_names = ['Insoluble', 'Membrane', 'Nucleus', 'Cytosol']

# تابع برای بارگذاری دنباله‌ها از فایل‌های FASTA
def load_sequences(fasta_files):
    sequences = []
    labels = []
    class_to_label = {'Insoluble': 0, 'Membrane': 1, 'Nucleus': 2, 'Cytosol': 3}

    for fasta_file in fasta_files:
        for record in SeqIO.parse(fasta_file, "fasta"):
            header = record.description
            class_name, _ = get_class_and_dist_from_header(header)
            if class_name is None:
                continue
            seq = str(record.seq)
            sequences.append(seq)
            labels.append(class_to_label[class_name])
    
    return sequences, np.array(labels)

# تابع برای استخراج کلاس و مقادیر dist از هدر
def get_class_and_dist_from_header(header):
    class_names = ['Insoluble', 'Membrane', 'Nucleus', 'Cytosol']
    try:
        dist_part = header.split("dist:")[1].split()[0]
        dist_values = list(map(float, dist_part.split("_")))
        max_index = np.argmax(dist_values)
        return class_names[max_index], dist_values
    except (IndexError, ValueError):
        return None, None

# فایل‌های FASTA
fasta_files = [
    "F:\\New Version\\Data\\cefra-seq\\cefra_seq_cDNA_ann_screened.fa",
    "F:\\New Version\\Data\\cefra-seq\\cefra_seq_cDNA_screened.fa"
]

# بارگذاری دنباله‌ها و برچسب‌ها
all_sequences, all_labels = load_sequences(fasta_files)

# بارگذاری ویژگی‌های تست
X_test_features = np.load(os.path.join(output_dir, 'X_test_features.npy'))
y_test_features = np.load(os.path.join(output_dir, 'y_test_features.npy'))

# بارگذاری مدل SVM
best_svm = joblib.load(os.path.join(output_dir, 'best_cefra_svm_model.pkl'))
print(f"مدل SVM از مسیر {os.path.join(output_dir, 'best_cefra_svm_model.pkl')} بارگذاری شد.")

# پیش‌بینی روی داده‌های تست
y_test_pred = best_svm.predict(X_test_features)

# محاسبه تعداد نوکلئوتیدهای درست پیش‌بینی‌شده برای هر کلاس
nucleotide_counts = {class_name: {'correct': 0, 'total': 0} for class_name in class_names}
sample_indices = []

# تقسیم داده‌ها به train و test مشابه کد اصلی
_, test_indices = train_test_split(range(len(all_sequences)), test_size=0.2, random_state=42, stratify=all_labels)

# بررسی پیش‌بینی‌ها
for idx, test_idx in enumerate(test_indices):
    true_label = y_test_features[idx]
    pred_label = y_test_pred[idx]
    seq = all_sequences[test_idx]
    seq_length = len(seq)

    class_name = class_names[true_label]
    nucleotide_counts[class_name]['total'] += seq_length
    if true_label == pred_label:
        nucleotide_counts[class_name]['correct'] += seq_length

    # ذخیره اطلاعات نمونه
    sample_indices.append({
        'index': idx,
        'sequence': seq[:10] + '...' if len(seq) > 10 else seq,
        'true_label': class_names[true_label],
        'pred_label': class_names[pred_label],
        'nucleotide_count': seq_length,
        'correct': 'Correct' if true_label == pred_label else 'Incorrect'
    })

# چاپ تعداد نوکلئوتیدهای درست پیش‌بینی‌شده
print("\nتعداد نوکلئوتیدهای درست پیش‌بینی‌شده برای هر کلاس:")
for class_name, counts in nucleotide_counts.items():
    correct = counts['correct']
    total = counts['total']
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"{class_name}: {correct} از {total} ({accuracy:.2f}%)")

# تبدیل نمونه‌ها به DataFrame
samples_df = pd.DataFrame(sample_indices)

# انتخاب 4 نمونه درست و 2 نمونه نادرست برای هر کلاس
selected_samples = []
for class_name in class_names:
    class_samples = samples_df[samples_df['true_label'] == class_name]
    correct_samples = class_samples[class_samples['correct'] == 'Correct'].head(4)
    incorrect_samples = class_samples[class_samples['correct'] == 'Incorrect'].head(2)
    selected_samples.extend(correct_samples.to_dict('records'))
    selected_samples.extend(incorrect_samples.to_dict('records'))

# تبدیل به DataFrame برای نمایش
selected_df = pd.DataFrame(selected_samples)

# رسم جدول بصری با بهبود گرافیکی
fig, ax = plt.subplots(figsize=(14, len(selected_df) * 0.6))
ax.axis('tight')
ax.axis('off')

# تنظیم رنگ برای ردیف‌ها (سبز برای درست، قرمز برای نادرست)
cell_colors = [['lightgreen' if row['correct'] == 'Correct' else 'lightcoral' for _ in range(5)] for row in selected_samples]

table = ax.table(cellText=selected_df[['sequence', 'true_label', 'pred_label', 'nucleotide_count', 'correct']].values,
                 colLabels=['Sequence (First 10 nt)', 'True Label', 'Predicted Label', 'Nucleotide Count', 'Status'],
                 cellLoc='center', colLoc='center', loc='center',
                 cellColours=cell_colors)
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1.5, 1.5)

# تنظیم حاشیه‌ها و عنوان
plt.title("Sample Predictions for Each Class", fontsize=14, pad=20)
for (row, col), cell in table.get_celld().items():
    cell.set_edgecolor('black')
    cell.set_linewidth(0.5)
    if row == 0:
        cell.set_text_props(weight='bold', color='darkblue')
    if col == 0:
        cell.set_text_props(ha='left')

plt.savefig(os.path.join(output_dir, 'sample_predictions_table.png'), bbox_inches='tight', dpi=300)
plt.close()
print(f"جدول نمونه‌ها با کیفیت بالا ذخیره شد: {os.path.join(output_dir, 'sample_predictions_table.png')}")